# Защита персональных данных клиентов в компании "Хоть потоп"
    
Мы работаем в страховой компании "Хоть потоп". Нам нужно защитить данные клиентов нашей страховой компании. Задача - разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, а также обосновать корректность его работы.

Предоставленная информация представляет собой датасет с признаками и целевым признаком. Признаки - пол, возраст и зарплата застрахованного, количество членов его семьи. Целевой признак - количество страховых выплат клиенту за последние 5 лет.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель нам не требуется.
    
____



В данной работе представлен процесс создания и обоснования метода преобразования признаков, в целях защиты персональной информации. Работа произведена в соответствии с инструкциями заказчика. Каждый значительный шаг сопровождается промежуточным, либо общим выводом.
  
    
    
____
    
    

    
Часть 1. Изучение общей информации о предоставленном датасете
    
1. Импорт необходимых библиотек для работы с данными, загрузка датасета
2. Изучение данных, получение общей информации
3. Вывод

    
Часть 2. Анализ изменения качества линейной регрессии

1. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
    
Часть 3. Создание алгоритма преобразования данных
    
1. Описание алгоритма
2. Обоснование алгоритма
    
Часть 4. Проверка алгоритма
    
1. Обучение моделей
2. Извлечение информации
3. Вывод

## Часть 1. Изучение общей информации о предоставленном датасете

<a id='Импорт необходимых библиотек для работы с данными, загрузка датасета'>

### <p style="text-align:center"> Импорт необходимых библиотек для работы с данными, загрузка датасета</p>

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.simplefilter('ignore')

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler 

Загружаем датасет, проверяем вывод.

In [4]:
df = pd.read_csv('/datasets/insurance.csv')

In [5]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<a id='Изучение данных, получение общей информации'></a>

### <p style="text-align:center"> Изучение данных, получение общей информации</p>

Ознакомимся с информацией, посмотрим соответствуют ли типы ячеек назначению.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [7]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Проверим наличие дубликатов.

In [8]:
df.duplicated().sum()

153

In [9]:
df = df.drop_duplicates()

Проверим наличие пропусков.

In [10]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

<a id='Вывод_1'></a>

### <p style="text-align:center"> Вывод</p>

##### В ходе первичного анализа данных были  сделаны следующие выводы:

___


1. Были рассмотрены основные характеристики предоставленного датасета.

2. Выявили наличие дубликатов, избавились от них.


___

Таким образом было выполнено первичное ознакомление с предоставленной информацией, выявленные дефекты устранены. Данные не нуждаются в предобработке. Переходим к основной части нашей работы.

## Часть 2. Анализ изменения качества линейной регрессии

<a id='Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?'>

### <p style="text-align:center"> Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? </p>

<b>Ответ</b>: не изменится, но ее придется обучить заново.

<b>Обоснование</b>: прямое. Будем считать, что утверждение верно, и попробуем найти противоречия.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула для расчета вектора весов линейной регрессии:
    
$$
w = (X^T X)^{-1} X^T y
$$

Представим матрицу признаков $X1$ как матрицу $X$, умноженную на обратимую матрицу $M$.

$$
w1 = ((XM)^T XM)^{-1} (XM)^T y
$$

Раскроем скобки.

$$
w1 = (M^TX^TXM)^{-1} M^TX^T y
$$

$$
w1 =  M^{-1}(X^TX)^{-1}(M^T)^{-1} M^TX^T y
$$

Так как матрица $M$ обратимая, произведение $(M^T)^{-1}$ на $M^T$ будет равно единичной матрице $E$.

$$
w1 = M^{-1}(X^TX)^{-1}EX^Ty
$$

$$
w1 =  M^{-1}w
$$

Подставим $X1$ и $w1$ в формулу для расчета предсказаний.

$$
a1 = X1w1 = XMM^{-1}w = XEw = a
$$

Таким образом, при умножении матрицы признаков на случайную обратимую матрицу, качество линейной регрессии не изменится.

## Часть 3. Создание алгоритма преобразования данных

<a id='Описание алгоритма'>

### <p style="text-align:center"> Описание алгоритма </p>

Как мы поняли из предыдущей части нашей работы, умножение признаков на обратимую матрицу не приведет к изменению предсказаний линейной регрессии. Соответственно, алгоритм будет заключаться в создании случайной обратимой матрицы $M$, с последующим умножением этой матрицы на матрицу признаков $X$, что приведет к своего рода кодированию данных без потери информативности. 
Качество моделей не должны отличаться, так как не будут отличаться предсказания. Попробуем обосновать это на примере.

<a id='Обоснование алгоритма'>

### <p style="text-align:center"> Обоснование алгоритма </p>

Создадим случайную матрицу. Размер 4х4 так как признаков 4, при умножении должна получиться матрица того же размера.

In [11]:
random_matrix = np.random.randint(100,size = (4, 4))
random_matrix

array([[93, 38, 31,  4],
       [48, 30, 54, 85],
       [83,  6,  7, 38],
       [63, 64,  1, 53]])

Проверяем, обратима ли наша матрица.

In [12]:
inverted_matrix = np.linalg.inv(random_matrix)
inverted_matrix

array([[ 0.0043988 , -0.00388279,  0.01072933, -0.0017976 ],
       [ 0.00623502, -0.00165727, -0.01686771,  0.01428115],
       [ 0.01309681,  0.0128574 , -0.0125245 , -0.01262896],
       [-0.01300495,  0.00637403,  0.00785113,  0.00399782]])

Отделим целевой признак.

In [13]:
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']

Для линейной регресии обычно производят масштабирование данных.

In [14]:
scaler = StandardScaler()
scaler.fit(features)
scaled_features = scaler.transform(features)

Теперь произведем расчет предсказаний с умножением и без умножения на обратимую матрицу.

In [15]:
w = np.linalg.inv(scaled_features.T.dot(scaled_features)).dot(scaled_features.T).dot(target)
a = scaled_features.dot(w)

А теперь произведем расчет предсказаний с умножением на обратимую матрицу.

In [16]:
encrypted_scaled_features = scaled_features.dot(random_matrix)
w1 = np.linalg.inv(encrypted_scaled_features.T.dot(encrypted_scaled_features)).dot(encrypted_scaled_features.T).dot(target)
a1 = encrypted_scaled_features.dot(w1)

Посмотрим насколько отличаются полученные предсказания.

In [17]:
(a - a1).sum()

-1.4164667702654121e-14

Полученные предсказания не отличаются друг от друга, качество одинаковое.

## Часть 4. Проверка алгоритма

<a id='Обучение моделей'>

### <p style="text-align:center"> Обучение моделей </p>

Разделим выборку на обучающие и тестовые данные. Проверим размеры полученных частей.

In [18]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=42)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((3635, 4), (1212, 4), (3635,), (1212,))

Также подготовим и закодированные данные. Проверим размеры полученных частей.

In [19]:
encrypted_scaled_features_train, encrypted_scaled_features_test = train_test_split(encrypted_scaled_features, random_state=42)
encrypted_scaled_features_train.shape, encrypted_scaled_features_test.shape

((3635, 4), (1212, 4))

In [20]:
clf_lr = LinearRegression()
clf_lr_encrypted = LinearRegression()

In [21]:
clf_lr.fit(features_train, target_train)
clf_lr_encrypted.fit(encrypted_scaled_features_train, target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
prediction = clf_lr.predict(features_test)
prediction_encrypted = clf_lr_encrypted.predict(encrypted_scaled_features_test)

In [23]:
r2 = r2_score(target_test, prediction)
mse = mean_squared_error(target_test, prediction)
r2_encrypted = r2_score(target_test, prediction_encrypted)
mse_encrypted = mean_squared_error(target_test, prediction_encrypted)
print(f'R2 score на исходных данных: {r2:.5f}')
print(f'MSE на исходных данных: {mse:.5f}')
print(f'R2 score на закодированных данных: {r2_encrypted:.5f}')
print(f'MSE на закодированных данных: {mse_encrypted:.5f}')

R2 score на исходных данных: 0.44346
MSE на исходных данных: 0.13415
R2 score на закодированных данных: 0.44346
MSE на закодированных данных: 0.13415


<a id='Извлечение информации'>

### <p style="text-align:center"> Извлечение информации </p>

Попробуем извлечь информицию из закодированного датафрейма.

In [24]:
encrypted_features = features.dot(random_matrix)
encrypted_features.head()

,0,1,2,3
0,4118924.0,298932.0,349446.0,1888342.0
1,3156271.0,229444.0,268485.0,1447963.0
2,1744392.0,126870.0,148566.0,800465.0
3,3462234.0,250958.0,293036.0,1586491.0
4,2167737.0,157478.0,184243.0,994184.0


In [25]:
excrypted_features = encrypted_features.dot(np.linalg.inv(random_matrix))
excrypted_features[4] = target
excrypted_features.columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты']
print('Извлеченная информация')
excrypted_features.round().astype('int64').head()

Извлеченная информация


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [26]:
print('Оригинальная информация')
df.head()

Оригинальная информация


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<a id='Вывод_2'>

### <p style="text-align:center"> Общий вывод </p>


___

Таким образом, в ходе работы над проектом в первую очередь было теоретически <a href='#Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?'>доказано</a>, что при умножении матрицы признаков на случайную обратимую матрицу, качество линейной регрессии не изменится; затем, был <a href='#Описание алгоритма'>описан</a> и <a href='#Обоснование алгоритма'>обоснован</a> алгоритм преобразования данных; в завершение были <a href='#Обучение моделей'>обучены</a> модели с оригинальными и закодированными признаками, а также произведено <a href='#Извлечение информации'>извлечение</a> информации из закодированного датафрейма. Метод подходит для шифрования персональной информации, при этом не ухудшает качество модели машинного обучения.